In [1]:
import numpy as np
import pandas as pd

data_path_train = 'Train_Dataset.csv'
data_path_test = 'Test_Dataset.csv'

dataset_train = pd.read_csv(data_path_train)
dataset_test = pd.read_csv(data_path_test)

dataset_train.drop(['Unnamed: 20'], axis = 1 , inplace =True)
dataset_test.drop(['Unnamed: 19','Unnamed: 20'], axis = 1 , inplace =True)

subset_train = dataset_train.columns.drop("customer_id")
duplicates_droped_dataset_train = dataset_train.drop_duplicates(subset = subset_train)

subset_test = dataset_test.columns.drop("customer_id")
duplicates_droped_dataset_test = dataset_test.drop_duplicates(subset = subset_test)

nan_added_dataset_train = duplicates_droped_dataset_train.copy()
nan_added_dataset_test = duplicates_droped_dataset_test.copy()

nan_added_dataset_train.loc[(nan_added_dataset_train['total_day_min']<0) &
                            (nan_added_dataset_train['total_day_calls']<0) & 
                            (nan_added_dataset_train['total_day_charge']<0) , 
                            ("total_day_min","total_day_calls",'total_day_charge')] = 0

nan_added_dataset_test.loc[(nan_added_dataset_test['total_day_min']<0) & 
                           (nan_added_dataset_test['total_day_calls']<0) & 
                           (nan_added_dataset_test['total_day_charge']<0) , 
                           ("total_day_min","total_day_calls",'total_day_charge')] = 0

for col in ['total_day_min','total_day_calls','total_day_charge', 'total_eve_min' ,'total_eve_calls',
            'total_eve_charge','total_night_minutes','total_night_calls','total_night_charge','total_intl_minutes','total_intl_calls',]:
    nan_added_dataset_train[col] = nan_added_dataset_train[col].abs()
    nan_added_dataset_test[col] = nan_added_dataset_test[col].abs()

    
odm_handled_dataset_train = nan_added_dataset_train.copy()
odm_handled_dataset_test = nan_added_dataset_test.copy()

for col in ['account_length','location_code']:
    odm_handled_dataset_train[col].fillna((odm_handled_dataset_train[col]).median(), inplace=True)
    odm_handled_dataset_test[col].fillna((odm_handled_dataset_test[col]).median(), inplace=True)
    
odm_handled_dataset_train.loc[(odm_handled_dataset_train['intertiol_plan'].isnull()), 'intertiol_plan'] = 'no'
odm_handled_dataset_test.loc[(odm_handled_dataset_test['intertiol_plan'].isnull()), 'intertiol_plan'] = 'no'

odm_handled_dataset_train.loc[(odm_handled_dataset_train['number_vm_messages']==0) & (odm_handled_dataset_train.voice_mail_plan.isnull()) , "voice_mail_plan"] = "no"
odm_handled_dataset_train.loc[(odm_handled_dataset_train['number_vm_messages']!=0) & (odm_handled_dataset_train.voice_mail_plan.isnull()) , "voice_mail_plan"] = "yes"

odm_handled_dataset_test.loc[(odm_handled_dataset_test['number_vm_messages']==0) & (odm_handled_dataset_test.voice_mail_plan.isnull()) , "voice_mail_plan"] = "no"
odm_handled_dataset_test.loc[(odm_handled_dataset_test['number_vm_messages']!=0) & (odm_handled_dataset_test.voice_mail_plan.isnull()) , "voice_mail_plan"] = "yes"


odm_handled_dataset_train.loc[(odm_handled_dataset_train['voice_mail_plan']=='no') & (odm_handled_dataset_train['number_vm_messages']!=0) , "number_vm_messages"] = 0
odm_handled_dataset_test.loc[(odm_handled_dataset_test['voice_mail_plan']=='no') & (odm_handled_dataset_test['number_vm_messages']!=0) , "number_vm_messages"] = 0


odm_handled_dataset_train.loc[(odm_handled_dataset_train.voice_mail_plan=='yes') & (odm_handled_dataset_train.number_vm_messages.isnull()), 'number_vm_messages'] = odm_handled_dataset_train.loc[odm_handled_dataset_train.voice_mail_plan=='yes', 'number_vm_messages'].median()
odm_handled_dataset_train.loc[(odm_handled_dataset_train.voice_mail_plan=='no') & (odm_handled_dataset_train.number_vm_messages.isnull()), 'number_vm_messages'] = 0

odm_handled_dataset_test.loc[(odm_handled_dataset_test.voice_mail_plan=='yes') & (odm_handled_dataset_test.number_vm_messages.isnull()), 'number_vm_messages'] = odm_handled_dataset_test.loc[odm_handled_dataset_test.voice_mail_plan=='yes', 'number_vm_messages'].median()
odm_handled_dataset_test.loc[(odm_handled_dataset_test.voice_mail_plan=='no') & (odm_handled_dataset_test.number_vm_messages.isnull()), 'number_vm_messages'] = 0

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_day_charge'])
odm_handled_dataset_train['total_day_min'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_day_min'])
odm_handled_dataset_train['total_day_charge'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['total_day_min','total_day_charge'])
odm_handled_dataset_train['total_day_calls'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_eve_charge'])
odm_handled_dataset_train['total_eve_min'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_eve_min'])
odm_handled_dataset_train['total_eve_charge'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['total_eve_min','total_eve_charge'])
odm_handled_dataset_train['total_eve_calls'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_night_charge'])
odm_handled_dataset_train['total_night_minutes'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_night_minutes'])
odm_handled_dataset_train['total_night_charge'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['total_night_minutes','total_night_charge'])
odm_handled_dataset_train['total_night_calls'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_intl_charge'])
odm_handled_dataset_train['total_intl_minutes'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['location_code', 'total_intl_minutes'])
odm_handled_dataset_train['total_intl_charge'].ffill(inplace=True)

odm_handled_dataset_train = odm_handled_dataset_train.sort_values(['total_intl_minutes','total_intl_charge'])
odm_handled_dataset_train['total_intl_calls'].ffill(inplace=True)

odm_handled_dataset_train['customer_service_calls'].fillna((odm_handled_dataset_train['customer_service_calls']).median(), inplace=True)

odm_handled_dataset_train.dropna(subset=['Churn'],inplace = True)


odm_handled_dataset_test= odm_handled_dataset_test.sort_values(['location_code', 'total_day_charge'])
odm_handled_dataset_test['total_day_min'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['location_code', 'total_day_min'])
odm_handled_dataset_test['total_day_charge'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['total_day_min','total_day_charge'])
odm_handled_dataset_test['total_day_calls'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['location_code', 'total_eve_charge'])
odm_handled_dataset_test['total_eve_min'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['location_code', 'total_eve_min'])
odm_handled_dataset_test['total_eve_charge'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['total_eve_min','total_eve_charge'])
odm_handled_dataset_test['total_eve_calls'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['location_code', 'total_night_charge'])
odm_handled_dataset_test['total_night_minutes'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['location_code', 'total_night_minutes'])
odm_handled_dataset_test['total_night_charge'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['total_night_minutes','total_night_charge'])
odm_handled_dataset_test['total_night_calls'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['location_code', 'total_intl_charge'])
odm_handled_dataset_test['total_intl_minutes'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['location_code', 'total_intl_minutes'])
odm_handled_dataset_test['total_intl_charge'].ffill(inplace=True)

odm_handled_dataset_test = odm_handled_dataset_test.sort_values(['total_intl_minutes','total_intl_charge'])
odm_handled_dataset_test['total_intl_calls'].ffill(inplace=True)

odm_handled_dataset_test['customer_service_calls'].fillna((odm_handled_dataset_test['customer_service_calls']).median(), inplace=True)

odm_handled_dataset_train.loc[(odm_handled_dataset_train['total_intl_calls']==0) & (odm_handled_dataset_train['total_intl_minutes']>0) & (odm_handled_dataset_train['total_intl_charge']>0) , ('total_intl_calls')] = 1
odm_handled_dataset_test.loc[(odm_handled_dataset_test['total_intl_calls']==0) & (odm_handled_dataset_test['total_intl_minutes']>0) & (odm_handled_dataset_test['total_intl_charge']>0) , ('total_intl_calls')] = 1

odm_handled_dataset_train = odm_handled_dataset_train.sort_index()
odm_handled_dataset_test = odm_handled_dataset_test.sort_index()

odm_handled_dataset_train.loc[odm_handled_dataset_train.total_day_min == 2283.9 , "total_day_min"] = 283.9
odm_handled_dataset_train.loc[odm_handled_dataset_train.total_eve_min == 5186.4 , "total_eve_min"] = 186.4
odm_handled_dataset_train.loc[odm_handled_dataset_train.total_night_minutes == 19700.0 , "total_night_minutes"] = 197.0
odm_handled_dataset_train.loc[odm_handled_dataset_train.total_day_calls == 10700.0 , "total_day_calls"] = 107.0
odm_handled_dataset_train.loc[odm_handled_dataset_train.total_night_charge == 900.15 , "total_night_charge"] = 9.15

pre_processed_dataset_train = odm_handled_dataset_train
pre_processed_dataset_test = odm_handled_dataset_test
pre_processed_dataset_train.to_csv("Cleaned_Train_Dataset.csv", index=False)
pre_processed_dataset_test.to_csv("Cleaned_Test_Dataset.csv", index=False)